# Import modules

In [10]:
import numpy as np
# Import visualization module for molecular visualization
# Import descriptors module to obtain descriptors for triad molecule
from tools.visualization import Visualization
from tools.descriptors import Descriptors

# Load files

In [2]:
file_dir = '/xspace/hl4212/DURF_datasets/triad_molecule'
traj_path = f'{file_dir}/triad_dataset.nc'
top_path = f'{file_dir}/triad_forcefield_ground.prmtop'

# Data visualization

In [4]:
triad_viz = Visualization(traj_path, top_path)
triad_viz.ball_and_stick()

NGLWidget(max_frame=99999)

## Compare different conformations

In [5]:
triad_viz.compare(0, -1)

NGLWidget()

# Get descriptors

In [8]:
d = Descriptors(traj_path, top_path)
EuclidianDist_1 = d.eucdist('C33','C128')
Angle_1 = d.angle('C33','C96','C128')
Angle_2 = d.angle('C33','C69','C96')
Angle_3 = d.angle('C69','C96','C128')
Dihedral_1 = d.dihedral('C21','C61','C66','C65')
Dihedral_2 = d.dihedral('C89','N6','C95','C96')
# For the RMSD descriptor, choose the frame with the largest 'Angle_1' as linear, the smallest as bent
# The frame number for linear is 88213, for bent is 29685
RMSD_1 = d.rmsd(frame = 88213)
RMSD_2 = d.rmsd(frame = 29685)

In [9]:
# Visulize the descriptors in dataframe
d_DataFrame = d.to_df(EuclidianDist_1=EuclidianDist_1, Angle_1=Angle_1,\
                      Angle_2=Angle_2, Angle_3=Angle_3, Dihedral_1=Dihedral_1,\
                      Dihedral_2=Dihedral_2, RMSD_1 =RMSD_1, RMSD_2 =RMSD_2)
d_DataFrame

,EuclidianDist_1,Angle_1,Angle_2,Angle_3,Dihedral_1,Dihedral_2,RMSD_1,RMSD_2
0,4.135001,1.862408,2.174203,1.523234,2.619274,3.074855,0.763624,1.046674
1,4.081912,1.844877,2.225012,1.549974,2.698790,3.141504,0.768220,1.047367
2,4.078993,1.791105,2.345837,1.558566,2.680488,-3.087373,0.775650,1.044716
3,4.120493,1.824511,2.289704,1.548207,2.608084,-2.905645,0.767361,1.058475
4,4.074001,1.811295,2.226435,1.545865,2.526079,-2.918643,0.784022,1.035676
...,...,...,...,...,...,...,...,...
99995,4.657602,2.134681,2.807392,2.029222,-2.145249,-3.141455,0.528814,1.204386
99996,4.543524,2.063565,2.831861,2.023469,-1.931827,2.972840,0.557411,1.186918
99997,4.593875,2.089307,2.805922,1.997702,-2.161717,2.770834,0.539554,1.194707
99998,4.625174,2.137845,2.851437,2.080499,-1.990369,3.005851,0.501717,1.232133


In [11]:
# Convert dataframe to nparray for dimensionality reduction
d_array = np.array(d_DataFrame)
d_array.shape

(100000, 8)